# Data preparation

In this notebook, we will demonstrate the data processing workflow of the `MassBalanceMachine` using an example with glaciological data from Icelandic glaciers. This example will guide you through the data processing pipeline, which retrieves topographical and meteorological features for all points with glaciological surface mass balance observations, and transforms the data to a monthly resolution.

We begin by importing some basic libraries along with the `massbalancemachine` library. Next, we specify the location where we will store the files for the region of interest. The data used in this example is from the [Icelandic Glaciers inventory](https://icelandicglaciers.is/index.html#/page/map), that was already preprocessed, to align with the WGMS data format we are using, in the [data preprocessing notebook](https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb). You can also download data from the [WGMS database](https://wgms.ch/data_databaseversions/), as you desire.

**Note:** WGMS data can contain errors or incorrect data, please check the data for correctness before using it.

**Note:** All data, stake measurements and glaciers outlines, are expected to have the WGS84 CRS.

In [ ]:
import os

repoPath = os.path.join(os.getcwd(), '../')
import pandas as pd
import geopandas as gpd

import massbalancemachine as mbm

cfg = mbm.Config()

cfg.month_list

## 1. Load your Target Surface Mass Balance Dataset and Retrieve RGI ID per Stake

In this step, we define and load our glaciological data from a region of interest. The WGMS dataset does not include RGI IDs by default, so we need to retrieve them from a glacier outline shapefile provided by the Randolph Glacier Inventory (v6). Each stake is then matched with an RGI ID. The RGI ID is necessary for the MassBalanceMachine to add additional topographical and meteorological features for the training stage.

The glacier outlines are automatically downloaded through OGGM as a shapefile.
It is also possible to provide your own shapefile directly (see example hereafter).

**Note:** Data records that have an invalid `FROM_DATE` or `TO_DATE`, where the day or month is indicated as 99, are deleted from the dataset.

In [ ]:
# Specify the filename of the input file with the raw data
target_data_fname = repoPath + 'notebooks/example_data/iceland/files/iceland_wgms_dataset.csv'

# Load the target data
data = pd.read_csv(target_data_fname)

### 1.1 Match the Stake Measurements with a RGI ID

Based on the location of the stake measurement given by `POINT_LAT` and `POINT_LON`, each data record is matched with the RGI ID for the glacier where the stake is located.

In [ ]:
# Get the RGI ID for each stake measurement for the region of interest
data = mbm.data_processing.utils.get_rgi(data=data, region=6)
data

In [ ]:
# # Specify the shape filename of the glaciers outline obtained from RGIv6
# glacier_outline_fname = 'path_to_your_own_shapefiles_folder/06_rgi60_Iceland.shp'
# # Load the glacier outlines
# glacier_outline = gpd.read_file(glacier_outline_fname)
# # Get the RGI ID for each stake measurement for the region of interest
# data = mbm.data_processing.utils.get_rgi(data=data, glacier_outlines=glacier_outline)
# data

Then, we can create a `Dataset`, by using the loaded dataframe for Iceland stake data together with the matched RGI IDs.

In [ ]:
# Provide the column name for the column that has the RGI IDs for each of the stakes
dataset = mbm.data_processing.Dataset(cfg,
                                      data=data,
                                      region_name='iceland',
                                      region_id=6,
                                      data_path=repoPath +
                                      'notebooks/example_data/iceland/files/')

## 2. Get Topographical Features per Stake Measurement

Once we have created a `Dataset`, the first thing we can do is to add topographical data in our dataset. This can be done automatically with `MassBalanceMachine`, which internally calls OGGM, by doing the following:

In [ ]:
# Specify the topographical features of interest
# Please see the OGGM documentation what variables are available: https://oggm.org/tutorials/stable/notebooks/10minutes/machine_learning.html ('topo', 'slope_factor', 'dis_from_border')
voi_topographical = ['aspect', 'slope']

# Retrieve the topographical features for each stake measurement based on the latitude and longitude of the stake and add them to the dataset
dataset.get_topo_features(vois=voi_topographical)

In [ ]:
df = dataset.data
df['MONTH_START'] = [str(date)[4:6] for date in df.FROM_DATE]
df['MONTH_END'] = [str(date)[4:6] for date in df.TO_DATE]
df.MONTH_START.unique(), df.MONTH_END.unique()

## 3. Get Meteorological Features per Stake Measurement

Once we have the topographical data, we can add the necessary climate data for the dataset. There are two ways to do so:
- manually download the dataset;
- or configure your CDS token to [use the CDSAPI](https://cds.climate.copernicus.eu/how-to-api) and `MassBalanceMachine` will automatically download the data for you.

### 3.1. Manually download ERA5 data

This is done by pulling that from ERA5-Land database for the region of interest. Before the climate data is matched with the stake measurements, we need to manually download the climate data for the region of interest from [Climate Copernicus website](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means?tab=form). Check the following options:


<table border="1" style="margin-top: 20px; margin-bottom: 20px; border-collapse: collapse; width: 100%;">
    <tr>
        <th style="padding: 10px; text-align: left;">Field</th>
        <th style="padding: 10px; text-align: left;">Details</th>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Product type:</strong></td>
        <td style="padding: 10px;"><em>Monthly averaged reanalysis</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Variable:</strong></td>
        <td style="padding: 10px;">
            <ul style="margin: 0; padding-left: 20px;">
                <li><em>2m temperature</em> (t2m)</li>
                <li><em>Total precipitation</em> (tp)</li>
                <li><em>Surface latent heat flux</em> (slhf)</li>
                <li><em>Surface sensible heat flux</em> (sshf)</li>
                <li><em>Surface solar radiation downwards</em> (ssrd)</li>
                <li><em>Forecast albedo</em> (fal)</li>
                <li><em>Surface net thermal radiation</em> (str)</li>
                <li>...</li>
                <li>You can explore additional options as you prefer. In this example, we use the variables listed above.</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Year:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Month:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Time:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Geographical area:</strong></td>
        <td style="padding: 10px;">
            <em>Either download for the entire Earth, or specify the coordinates for a bounding box in the region of interest. For the region of interest, provide a North, East, South, and West coordinate, specifying the two corners of the bounding box.</em>
        </td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Format:</strong></td>
        <td style="padding: 10px;"><em>NetCDF-3</em></td>
    </tr>
</table>


Then click _Submit Request_ (after you have registered or logged into your account). Please be aware that downloading this data can take up to several hours, depending on the number of variables, timescales, and the area selected. Once the download is complete, place the netCDF file in the correct directory and rename it accordingly.

Additionally, we need the _geopotential height_ as an extra variable in our dataset. We will calculate a new variable by determining the difference between the geopotential height and the elevation at the stake measurement. The purpose of this height difference is to encourage the model to use it for downscaling the ERA5Land climate data, rather than relying on the lapse rate. The geopotential is downloaded from [here](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation#ERA5Land:datadocumentation-parameterlistingParameterlistings), and is already included in this example.

In [ ]:
# Specify the files of the climate data, that will be matched with the coordinates of the stake data
era5_climate_data = repoPath + 'notebooks/example_data/iceland/climate/era5_monthly_averaged_data.nc'
geopotential_data = repoPath + 'notebooks/example_data/iceland/climate/era5_geopotential_pressure.nc'

# Match the climate features, from the ERA5Land netCDF file, for each of the stake measurement dataset
dataset.get_climate_features(climate_data=era5_climate_data,
                             geopotential_data=geopotential_data)

### 3.2. Automatically download ERA5 data using the CDSAPI

This example is commented below since this can take some time to download and you need to create valid tokens on the [CDS website](https://cds.climate.copernicus.eu/how-to-api).

In [ ]:
# Match the climate features, from the ERA5Land netCDF file, for each of the stake measurement dataset
# dataset.get_climate_features()

Climate data for each stake position is retrieved for each month of the hydrological year (1 Oct - 30 Sept) given by `YEAR`.

## 4. Transform Data to Monthly Time Resolution

Finally, we need to transform the dataset into a monthly resolution, accounting for a variable period (`FROM_DATE` and `TO_DATE`) for the SMB target data. This will be done in order to perform SMB predictions at a monthly time step, which then will be integrated both in time and space to match the available glaciological and geodetic SMB observations for the training. Please specify the climate variables used in the previous step in the list below. Consult the [ERA5 Land documentation](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation) of all short names of the climate variables.

When accounting for variable periods `MassBalanceMachine` uses the closest start of month to `FROM_DATE` and `TO_DATE` and includes all months between. For example, `FROM_DATE 19940917` and `TO_DATE 19950520` will include all months from `October 1994` through `May 1995`.

In [ ]:
# Specify the short names of the climate variables available in the dataset
vois_climate = ['t2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str']

# For each record, convert to a monthly time resolution
dataset.convert_to_monthly(vois_climate=vois_climate,
                           vois_topographical=voi_topographical)

Finally, we can take a look at the dataset which will be used for training.

In [ ]:
display(dataset.data)

We have finished preprocessing the training data for our machine learning model. You can either explore the dataset in this [notebook](https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/date_exploration.ipynb) or continue with the [XGBoost model training notebook](https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/model_training_xgboost.ipynb).